In [1]:
from model import atp_graph, losses
from data_wrangler import synthetic_data_gen, feature_extractor
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from model import atp_pipeline
from comparison_models.tnp import tnp_pipeline
from data_wrangler import dataset_preparer
import argparse
from data_wrangler.batcher import batcher, batcher_np
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
exchange = pd.read_csv('datasets/exchange.csv')

In [3]:
exchange

,date,0,1,2,3,4,5,6,OT
0,1990/1/1 0:00,0.785500,1.611000,0.861698,0.634196,0.211242,0.006838,0.525486,0.593000
1,1990/1/2 0:00,0.781800,1.610000,0.861104,0.633513,0.211242,0.006863,0.523972,0.594000
2,1990/1/3 0:00,0.786700,1.629300,0.861030,0.648508,0.211242,0.006975,0.526316,0.597300
3,1990/1/4 0:00,0.786000,1.637000,0.862069,0.650618,0.211242,0.006953,0.523834,0.597000
4,1990/1/5 0:00,0.784900,1.653000,0.861995,0.656254,0.211242,0.006940,0.527426,0.598500
...,...,...,...,...,...,...,...,...,...
7583,2010/10/6 0:00,0.718494,1.222195,0.737485,0.969974,0.143697,0.008500,0.688565,0.690846
7584,2010/10/7 0:00,0.721839,1.223459,0.741155,0.977297,0.143763,0.008595,0.690288,0.695701
7585,2010/10/8 0:00,0.723197,1.234111,0.745184,0.984446,0.143997,0.008562,0.691419,0.695943
7586,2010/10/9 0:00,0.720825,1.233905,0.744131,0.980344,0.143993,0.008555,0.690942,0.692689


In [5]:
ex_array[:, 0] = np.linspace(0, 1, len(ex_array[:, 0]))

In [13]:
ex_array = np.float32(ex_array)

In [103]:
def batcher_bc(array, n_C, n_T, batch_size=32):
    eye_matrix = np.eye(array.shape[1]-1)
    y_reordered = []
    t_reordered = []
    t = []
    y = []
    for i in range(batch_size):
        index = int(np.random.randint(0, array.shape[0] - n_C - n_T, 1))
        t_context =  array[index:index + n_C , 0]
        t_context = np.repeat(t_context, array.shape[1] - 1)[:, np.newaxis]
        t_target = array[index + n_C:index + n_C + n_T, 0]
        t_target = np.repeat(t_target, array.shape[1] - 1)[:, np.newaxis]
        
        y_temp_context = array[index:index + n_C , 1]
        y_temp_target = array[index + n_C:index + n_C + n_T, 1]
        y_temp_all = np.concatenate([y_temp_context, y_temp_target], axis=0)
        t_all = np.repeat(np.concatenate([t_context, t_target], axis=0), array.shape[1] - 1)
        
        y_temp_context = np.concatenate([y_temp_context[:, np.newaxis], np.repeat(eye_matrix[:, 0][np.newaxis, :], repeats=n_C, axis=0)], axis=1)
        y_temp_target = np.concatenate([y_temp_target[:, np.newaxis], np.repeat(eye_matrix[:, 0][np.newaxis, :], repeats=n_T, axis=0)], axis=1)
        for i in range(2, array.shape[1]):
            y_next_context = np.concatenate([array[index:index + n_C, i][:, np.newaxis], np.repeat(eye_matrix[:, i - 1][np.newaxis, :], repeats=n_C, axis=0)], axis=1)
            y_temp_context = np.concatenate([y_temp_context, y_next_context], axis=0)
            y_next_target  = np.concatenate([array[index + n_C:index + n_C + n_T, i][:, np.newaxis], np.repeat(eye_matrix[:, i - 1][np.newaxis, :], repeats=n_T, axis=0)], axis=1)
            y_temp_target  = np.concatenate([y_temp_target, y_next_target], axis=0)
            y_temp_all = np.concatenate([y_temp_all, array[index:index + n_C + n_T, i]], axis=0)


        y_temp = np.concatenate([y_temp_context, y_temp_target], axis=0)
        y_reordered.append(y_temp)
        t_reordered.append(np.concatenate([t_context, t_target], axis=0))
        t.append(t_all)
        y.append(y_temp_all)
    return np.array(y), np.array(y_reordered), np.array(t), np.array(t_reordered)

In [104]:
y, y_reordered, t, t_reordered = batcher_bc(ex_array, 20, 10, 32)


In [105]:
t_reordered.shape

(32, 240, 1)

In [106]:
n_C = 20
n_T = 10

In [77]:
n_C_b  = n_C * (y.shape[-1] - 1)
n_T_b = n_T * (y.shape[-1] - 1)

In [78]:
        context_part = tf.concat([tf.ones((n_C_b,n_C_b),tf.bool),tf.zeros((n_C_b,n_T_b),tf.bool)],axis=-1)
        diagonal_mask = tf.linalg.band_part(tf.ones((n_C_b+n_T_b,n_C_b+n_T_b),tf.bool),-1,0)
        lower_diagonal_mask = tf.linalg.set_diag(diagonal_mask,tf.zeros(diagonal_mask.shape[0:-1],tf.bool)) ### condense into one line?                                                                               
        mask = tf.concat([context_part,lower_diagonal_mask[n_C_b:n_C_b+n_T_b,:n_C_b+n_T_b]],axis=0) # check no conflicts with init and check mask is correct shape

In [112]:
class DE(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.batch_norm_layer = tf.keras.layers.BatchNormalization()

    def call(self,  inputs):
        y,  x,  n_C,  n_T,  training = inputs
        print(x.shape)
        if (x.shape[-1] == 1):
            y_diff,  x_diff,  d,  x_n,  y_n = self.derivative_function([y,  x,  n_C,  n_T])
        else: 
            y_diff,  x_diff,  d,  x_n,  y_n = self.derivative_function_2d([y,  x,  n_C,  n_T])

        d_1 = tf.where(tf.math.is_nan(d),  10000.0,  d)
        d_2 = tf.where(tf.abs(d) > 200.,  0.,  d)
        d = self.batch_norm_layer(d_2,  training=training)

        d_label = tf.cast(tf.math.equal(d_2,  d_1),  "float32")
        d = tf.concat([d,  d_label],  axis=-1)

        return y_diff,  x_diff,  d,  x_n,  y_n


###### i think here what we do is calculate the derivative at the given y value and add that in as a feature. This is masked when making predictions
# so the derivative of other y values are what are seen
#  Based on taylor expansion, a better feature would be including the derivative of the closest x point, where only seen y values are used for the differencing. 
#this derivative wouldn't need masking.

 ############ check what to do for 2d derivatives - should y diff just be for one point? for residual trick that would make most sense.
 #### but you need mutli-dimensional y for the derivative

 ###### and explain why we do this

    def derivative_function(self,  inputs, ϵ=0.000002):
        ## need to add shape information to inputs
        print('please add shape information to inputs')
        
        y_values,  x_values,  n_C,  n_T = inputs

        batch_size = y_values.shape[0]

        dim_x = x_values.shape[-1]
        dim_y = y_values.shape[-1]


        #context section

        current_x = tf.expand_dims(x_values[:, :n_C], axis=2)
        current_y = tf.expand_dims(y_values[:, :n_C], axis=2)

        x_temp = x_values[:, :n_C]
        x_temp = tf.repeat(tf.expand_dims(x_temp,  axis=1),  axis=1,  repeats=n_C)

        y_temp = y_values[:, :n_C]
        y_temp = tf.repeat(tf.expand_dims(y_temp,  axis=1),  axis=1,  repeats=n_C)

        ix = tf.argsort(tf.math.reduce_euclidean_norm((current_x - x_temp), axis=-1), axis=-1)[:, :, 1]        
        selection_indices = tf.concat([tf.reshape(tf.repeat(tf.range(batch_size*n_C), 1), (-1, 1)), 
                                       tf.reshape(ix, (-1, 1))], axis=1)


        x_closest = tf.reshape(tf.gather_nd(tf.reshape(x_temp, (-1, n_C, dim_x)), selection_indices), 
                               (batch_size, n_C, dim_x)) 
        
        
        y_closest = tf.reshape(tf.gather_nd(tf.reshape(y_temp, (-1, n_C, dim_y)), selection_indices), 
                       (batch_size, n_C, dim_y))
        
        x_rep = current_x[:, :, 0] - x_closest
        y_rep = current_y[:, :, 0] - y_closest            

        deriv = y_rep / (ϵ + tf.math.reduce_euclidean_norm(x_rep, axis=-1, keepdims=True))

        dydx_dummy = deriv
        diff_y_dummy = y_rep
        diff_x_dummy =x_rep
        closest_y_dummy = y_closest
        closest_x_dummy = x_closest

        #target selection

        current_x = x_values[:, n_C:n_C+n_T, tf.newaxis]
        current_y = y_values[:, n_C:n_C+n_T, tf.newaxis]

        x_temp = tf.repeat(x_values[:, :n_T+n_C], axis=1, repeats=n_T)
        y_temp = tf.repeat(y_values[:, :n_T+n_C], axis=1, repeats=n_T)


        x_mask = tf.linalg.band_part(tf.ones((n_T, n_C + n_T), tf.bool), -1, n_C)
        x_mask_inv = (x_mask == False)
        x_mask_float = tf.cast(x_mask_inv, "float32")*1000
        x_mask_float_repeat = tf.repeat(x_mask_float[tf.newaxis, :], axis=0, repeats=batch_size)

        print((current_x - x_temp).shape)

        # print(x_mask_repeat.shape)
        ix = tf.argsort(tf.cast(tf.math.reduce_euclidean_norm((current_x - x_temp), 
                                            axis=-1), dtype="float32") + x_mask_float_repeat, axis=-1)[:, :, 1]

        print(ix.shape)
        selection_indices = tf.concat([tf.reshape(tf.repeat(tf.range(batch_size*n_T), 1), (-1, 1)), 
                                   tf.reshape(ix, (-1, 1))], axis=1)

        x_closest = tf.reshape(tf.gather_nd(tf.reshape(x_temp, (-1, n_T+n_C, dim_x)), selection_indices), 
                               (batch_size, n_T, dim_x)) 
        
        y_closest = tf.reshape(tf.gather_nd(tf.reshape(y_temp, (-1, n_T+n_C, dim_y)), selection_indices), 
                       (batch_size, n_T, dim_y))
        
        
        x_rep = current_x[:, :, 0] - x_closest
        y_rep = current_y[:, :, 0] - y_closest            

        deriv = y_rep / (ϵ + tf.math.reduce_euclidean_norm(x_rep, axis=-1, keepdims=True))

        dydx_dummy = tf.concat([dydx_dummy, deriv], axis=1)
        diff_y_dummy = tf.concat([diff_y_dummy, y_rep], axis=1)
        diff_x_dummy = tf.concat([diff_x_dummy, x_rep], axis=1)
        closest_y_dummy = tf.concat([closest_y_dummy, y_closest], axis=1)
        closest_x_dummy = tf.concat([closest_x_dummy, x_closest], axis=1)

        return diff_y_dummy, diff_x_dummy, dydx_dummy, closest_x_dummy, closest_y_dummy


    def derivative_function_2d(self, inputs,  ϵ = 0.0000):

        
            def dydz(current_y, y_closest_1, y_closest_2, current_x, x_closest_1, x_closest_2):
                #"z" is the second dim of x input
                numerator = y_closest_2 - current_y[:, :, 0] - ((x_closest_2[:, :, :1]-current_x[:, :, 0, :1])*(y_closest_1-current_y[:, :, 0] ))/(x_closest_1[:, :, :1]-current_x[:, :, 0, :1] +ϵ)
                denom = x_closest_2[:, :, 1:2] - current_x[:, :, 0, 1:2] - (x_closest_1[:, :, 1:2]-current_x[:, :, 0, 1:2])*(x_closest_2[:, :, :1]-current_x[:, :, 0, :1])/(x_closest_1[:, :, :1]-current_x[:, :, 0, :1]+ϵ)
                dydz_pred = numerator/(denom+ϵ)
                return dydz_pred
            
            def dydx(dydz, current_y, y_closest_1, current_x, x_closest_1):
                dydx = (y_closest_1-current_y[:, :, 0] - dydz*(x_closest_1[:, :, 1:2]-current_x[:, :, 0, 1:2]))/(x_closest_1[:, :, :1]-current_x[:, :, 0, :1]+ϵ)
                return dydx

            y_values, x_values, n_C, n_T = inputs

            batch_size, length = y_values.shape[0], n_C + n_T

            dim_x = x_values.shape[-1]
            dim_y = y_values.shape[-1]


            #context section

            current_x = tf.expand_dims(x_values[:, :n_C], axis=2)
            current_y = tf.expand_dims(y_values[:, :n_C], axis=2)

            x_temp = x_values[:, :n_C]
            x_temp = tf.repeat(tf.expand_dims(x_temp, axis=1), axis=1, repeats=n_C)

            y_temp = y_values[:, :n_C]
            y_temp = tf.repeat(tf.expand_dims(y_temp, axis=1), axis=1, repeats=n_C)

            ix_1 = tf.argsort(tf.math.reduce_euclidean_norm((current_x - x_temp), axis=-1), axis=-1)[:, :, 1]        
            selection_indices_1 = tf.concat([tf.reshape(tf.repeat(tf.range(batch_size*n_C), 1), (-1, 1)), 
                                                tf.reshape(ix_1, (-1, 1))], axis=1)

            ix_2 = tf.argsort(tf.math.reduce_euclidean_norm((current_x - x_temp), axis=-1), axis=-1)[:, :, 2]        
            selection_indices_2 = tf.concat([tf.reshape(tf.repeat(tf.range(batch_size*n_C), 1), (-1, 1)), 
                                        tf.reshape(ix_2, (-1, 1))], axis=1)


            x_closest_1 = tf.reshape(tf.gather_nd(tf.reshape(x_temp, (-1, n_C, dim_x)), selection_indices_1), 
                                (batch_size, n_C, dim_x)) +   tf.random.normal(shape=(batch_size,  n_C,  dim_x), stddev=0.01)

            x_closest_2 = tf.reshape(tf.gather_nd(tf.reshape(x_temp, (-1, n_C, dim_x)), selection_indices_2), 
                                (batch_size, n_C, dim_x)) +   tf.random.normal(shape=(batch_size, n_C, dim_x), stddev=0.01)



            y_closest_1 = tf.reshape(tf.gather_nd(tf.reshape(y_temp, (-1, n_C, dim_y)), selection_indices_1), 
                        (batch_size, n_C, dim_y))


            y_closest_2 = tf.reshape(tf.gather_nd(tf.reshape(y_temp, (-1, n_C, dim_y)), selection_indices_2), 
                        (batch_size, n_C, dim_y))


            x_rep_1 = current_x[:, :, 0] - x_closest_1
            x_rep_2 = current_x[:, :, 0] - x_closest_2

            print(y_closest_1.shape, current_y.shape)
            y_rep_1 = current_y[:, :, 0] - y_closest_1
            y_rep_2 = current_y[:, :, 0] - y_closest_2

            dydx_2 = dydz(current_y, y_closest_1, y_closest_2, current_x, x_closest_1, x_closest_2)
            dydx_1 = dydx(dydx_2, current_y, y_closest_1, current_x, x_closest_1)

            deriv_dummy = tf.concat([dydx_1, dydx_2], axis=-1)

            diff_y_dummy = tf.concat([y_rep_1, y_rep_2], axis=-1)

            diff_x_dummy =tf.concat([x_rep_1, x_rep_2], axis=-1)

            closest_y_dummy = tf.concat([y_closest_1, y_closest_2], axis=-1)
            closest_x_dummy = tf.concat([x_closest_1, x_closest_2], axis=-1)

            #target selection

            current_x = tf.expand_dims(x_values[:, n_C:n_C+n_T], axis=2)
            current_y = tf.expand_dims(y_values[:, n_C:n_C+n_T], axis=2)

            x_temp = tf.repeat(tf.expand_dims(x_values[:, :n_T+n_C], axis=1), axis=1, repeats=n_T)
            y_temp = tf.repeat(tf.expand_dims(y_values[:, :n_T+n_C], axis=1), axis=1, repeats=n_T)

            
            x_mask = 1000  * (1 -  tf.linalg.band_part(tf.ones((n_T, n_C + n_T), tf.int32), -1, n_C))
            x_mask_repeat = tf.repeat(x_mask[tf.newaxis, :], axis=0, repeats=batch_size)
            
            ix_1 = tf.argsort(tf.cast(tf.math.reduce_euclidean_norm((current_x - x_temp), 
                                                axis=-1), dtype="float32") + x_mask_repeat, axis=-1)[:, :, 1]
            selection_indices_1 = tf.concat([tf.reshape(tf.repeat(tf.range(batch_size*n_T), 1), (-1, 1)), 
                                                tf.reshape(ix_1, (-1, 1))], axis=1)
            
            
            
            ix_2 = tf.argsort(tf.cast(tf.math.reduce_euclidean_norm((current_x - x_temp), 
                                                axis=-1), dtype="float32") + x_mask_repeat, axis=-1)[:, :, 2]
            selection_indices_2 = tf.concat([tf.reshape(tf.repeat(tf.range(batch_size*n_T), 1), (-1, 1)), 
                                                tf.reshape(ix_2, (-1, 1))], axis=1)
            
            
            
            x_closest_1 = tf.reshape(tf.gather_nd(tf.reshape(x_temp, (-1, n_T+n_C, dim_x)), selection_indices_1), 
                                (batch_size, n_T, dim_x)) +   tf.random.normal(shape=(batch_size, n_T, dim_x), stddev=0.01)

            x_closest_2 = tf.reshape(tf.gather_nd(tf.reshape(x_temp, (-1, n_T+n_C, dim_x)), selection_indices_2), 
                                (batch_size, n_T, dim_x)) +   tf.random.normal(shape=(batch_size, n_T, dim_x), stddev=0.01)



            y_closest_1 = tf.reshape(tf.gather_nd(tf.reshape(y_temp, (-1, n_T+n_C, dim_y)), selection_indices_1), 
                        (batch_size, n_T, dim_y))


            y_closest_2 = tf.reshape(tf.gather_nd(tf.reshape(y_temp, (-1, n_T+n_C, dim_y)), selection_indices_2), 
                        (batch_size, n_T, dim_y))
        

            x_rep_1 = current_x[:, :, 0] - x_closest_1
            x_rep_2 = current_x[:, :, 0] - x_closest_2

            y_rep_1 = current_y[:, :, 0] - y_closest_1
            y_rep_2 = current_y[:, :, 0] - y_closest_2

            dydx_2 = dydz(current_y, y_closest_1, y_closest_2, current_x, x_closest_1, x_closest_2)
            dydx_1 = dydx(dydx_2, current_y, y_closest_1, current_x, x_closest_1)
            
            deriv_dummy_2 = tf.concat([dydx_1, dydx_2], axis=-1)

            diff_y_dummy_2 = tf.concat([y_rep_1, y_rep_2], axis=-1)

            diff_x_dummy_2 =tf.concat([x_rep_1, x_rep_2], axis=-1)

            closest_y_dummy_2 = tf.concat([y_closest_1, y_closest_2], axis=-1)
            closest_x_dummy_2 = tf.concat([x_closest_1, x_closest_2], axis=-1)
            
            ########## concat all ############


            deriv_dummy_full = tf.concat([deriv_dummy, deriv_dummy_2], axis=1)
            diff_y_dummy_full = tf.concat([diff_y_dummy, diff_y_dummy_2], axis=1)
            diff_x_dummy_full = tf.concat([diff_x_dummy, diff_x_dummy_2], axis=1)
            closest_y_dummy_full = tf.concat([closest_y_dummy, closest_y_dummy_2], axis=1)
            closest_x_dummy_full = tf.concat([closest_x_dummy, closest_x_dummy_2], axis=1)

            return diff_y_dummy_full, diff_x_dummy_full, deriv_dummy_full, closest_x_dummy_full, closest_y_dummy_full




In [113]:
dd = DE()

In [114]:
mask

<tf.Tensor: shape=(240, 240), dtype=bool, numpy=
array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ...,  True, False, False],
       [ True,  True,  True, ...,  True,  True, False]])>

In [119]:
t.shape

(32, 1920)

In [116]:
y.shape

(32, 240)

In [121]:
for i in range(y.shape[1] - 1):
    y_diff, x_diff, d, x_n, y_n = dd([y[i*(n_C + n_T):(i+1)*(n_C + n_T)], t[i*(n_C + n_T):(i+1)*(n_C + n_T), 0][:, np.newaxis], n_C, n_T, True])


(30, 1)
please add shape information to inputs


2023-05-10 21:26:34.626375: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at strided_slice_op.cc:108 : INVALID_ARGUMENT: Index out of range using input dim 2; input has only 2 dims


InvalidArgumentError: Exception encountered when calling layer "de_1" (type DE).

Index out of range using input dim 2; input has only 2 dims [Op:StridedSlice] name: de_1/strided_slice/

Call arguments received:
  • inputs=['tf.Tensor(shape=(30, 240), dtype=float32)', 'tf.Tensor(shape=(30, 1), dtype=float32)', 'tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=bool)']